In [1]:
import numpy as np
from datetime import datetime

import keras
import scipy
from keras import backend as K
from keras.preprocessing import image
from keras.applications import inception_v3

Using TensorFlow backend.


In [ ]:
K.set_learning_phase(0)
# VGG16,VGG19,Xception,ResNet50 등 다양한 컨브넷으로 사용 할 수 있다.
model = inception_v3.InceptionV3(weights='imagenet',
                                 include_top=False)
# 레이어층을 다양하게 설정 할 수 있다.
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}
layer_dict = dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    
    activation = layer_dict[layer_name].output

    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling\
# 이 텐서는 생성된 딥드림 이미지를 저장
dream = model.input
grads = K.gradients(loss, dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)
def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...', i, '번째 손실 :', loss_value)
        x += step * grad_values
    return x
def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)
def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    image.save_img(fname, pil_img)
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x
#####################3# 하이퍼파라미터를 바꾸면 새로운 효과가 만들어집니다 #############################
step = 0.01  # 경상 상승법 단계 크기 -> 0 ~ 1
num_octave = 3  # 경사 상승법을 실행할 스케일 단계 횟수 ->
octave_scale = 1.4  # 스케일 간의 크기 비율 -> 크기
iterations = 20  # 스케일 단계마다 수행할 경사 상승법 횟수 ->

max_loss = 10. # -> 사진 꺠지는걸 방지 할 수 있음. 너무 작으면 효과가 적음

# 사용할 이미지 경로 입력
base_image_path = '/home/kiho/바탕화면/sample.jpg'

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

successive_shapes = successive_shapes[::-1]
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('처리할 이미지 크기', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

date_time = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
save_img(img, fname=f'/home/kiho/바탕화면/{date_time}.jpg')

Instructions for updating:
Colocations handled automatically by placer.
처리할 이미지 크기 (20, 17)


'2019-12-26-21-03-26'